Our task is to build a consultant bot that can answer questions of different domains, such as medical with a doctor bot or legal with a lawyer bot.

We will show how flexible ``Component`` and the ``Sequential`` container is to build the same task
in different ways.

1. **Single Task**: We can build a single task where it deals with multiple generators and handles any coding logic.
2. **Multiple Tasks** and combine them using ``Sequential`` which resembles the concept of `Chain` or pipelines in other libraries.


First, lets prepare the imports and prompt templates using `jinjia2` template. We plan to demonstrate how we can use different models too. If this tutorial is the first thing you read, no need to care about more details, but focus on how the `development process` looks like using `LightRAG` library.

In [1]:
import re
from lightrag.core import Component, Generator, Sequential
from lightrag.components.model_client import OpenAIClient
from lightrag.components.model_client import GroqAPIClient
from lightrag.utils import setup_env # make sure you have a .env file with OPENAI_API_KEY and GROQ_API_KEY

In [2]:
template_doc = r"""<SYS> You are a doctor </SYS> User: {{input_str}}"""
template_law = r"""<SYS> You are a lawyer </SYS> User: {{input_str}}"""
template_router = r"""<SYS> You are a router who will route a user question to the right generator.
            Here are your choices in form of key: value pairs:
             {% for key, value in choices.items() %}
                {{ key }}: {{ value }}
             {% endfor %}
            Output the key of your choice.
            </SYS> User question: {{input_str}}
            You:
            """

Let's turn on the library log to help with debugging.

In [3]:
from lightrag.utils import enable_library_logging
enable_library_logging()

In [4]:
#Toy example

class DocQA(Component):
    def __init__(self):
        super(DocQA, self).__init__()
        self.doc = Generator(
            template=template_doc,
            model_client=OpenAIClient(),
            model_kwargs={"model": "gpt-3.5-turbo"},
        )

    def call(self, query: str) -> str:
        return self.doc(prompt_kwargs={"input_str": query}).data
    

In [5]:
doc = DocQA()
doc

2024-06-09 20:25:20 - INFO - [prompt_builder.py:82:__init__] - Prompt has variables: ['input_str']


DocQA(
  (doc): Generator(
    model_kwargs={'model': 'gpt-3.5-turbo'}, model_type=ModelType.LLM
    (prompt): Prompt(template: <SYS> You are a doctor </SYS> User: {{input_str}}, prompt_variables: ['input_str'])
    (model_client): OpenAIClient()
  )
)

In [6]:
print(doc("What is the best treatment for headache?"))

2024-06-09 20:25:20 - INFO - [generator.py:196:call] - prompt_kwargs: {'input_str': 'What is the best treatment for headache?'}
2024-06-09 20:25:20 - INFO - [generator.py:197:call] - model_kwargs: {}
2024-06-09 20:25:20 - INFO - [openai_client.py:122:call] - api_kwargs: {'model': 'gpt-3.5-turbo', 'messages': [{'role': 'system', 'content': '<SYS> You are a doctor </SYS> User: What is the best treatment for headache?'}]}
2024-06-09 20:25:22 - INFO - [_client.py:1026:_send_single_request] - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-06-09 20:25:22 - INFO - [generator.py:205:call] - output: GeneratorOutput(data="As a doctor, I would recommend trying over-the-counter pain relievers such as acetaminophen, ibuprofen, or aspirin. It's important to stay well-hydrated, get plenty of rest, and practice relaxation techniques such as deep breathing or meditation. If the headache persists or worsens, it's important to consult with a healthcare provider for f

In [7]:
# list other subcomponents

for subcomponent in doc.named_components():
    print(subcomponent)

('', DocQA(
  (doc): Generator(
    model_kwargs={'model': 'gpt-3.5-turbo'}, model_type=ModelType.LLM
    (prompt): Prompt(template: <SYS> You are a doctor </SYS> User: {{input_str}}, prompt_variables: ['input_str'])
    (model_client): OpenAIClient()
  )
))
('doc', Generator(
  model_kwargs={'model': 'gpt-3.5-turbo'}, model_type=ModelType.LLM
  (prompt): Prompt(template: <SYS> You are a doctor </SYS> User: {{input_str}}, prompt_variables: ['input_str'])
  (model_client): OpenAIClient()
))
('doc.prompt', Prompt(template: <SYS> You are a doctor </SYS> User: {{input_str}}, prompt_variables: ['input_str']))
('doc.model_client', OpenAIClient())


Let's add a parameter

In [8]:
from lightrag.core.parameter import Parameter

doc.register_parameter("demo", param=Parameter(data="demo"))

In [9]:
# list all parameters
for param in doc.named_parameters():
    print(param)

('demo', Parameter: demo)


In [10]:
doc.to_dict()

{'type': 'DocQA',
 'data': {'_components': {'doc': {'type': 'Generator',
    'data': {'_components': {'model_client': {'type': 'OpenAIClient',
       'data': {'_components': {},
        '_parameters': {},
        'training': False,
        'sync_client': <openai.OpenAI at 0x1191fc250>,
        'async_client': None,
        '_api_key': None}},
      'prompt': {'type': 'Prompt',
       'data': {'_components': {},
        '_parameters': {},
        'training': False,
        '_template_string': '<SYS> You are a doctor </SYS> User: {{input_str}}',
        'prompt_variables': ['input_str'],
        'preset_prompt_kwargs': None}}},
     '_parameters': {},
     'training': False,
     'template': '<SYS> You are a doctor </SYS> User: {{input_str}}',
     'preset_prompt_kwargs': None,
     'model_kwargs': {'model': 'gpt-3.5-turbo'},
     'output_processors': None,
     '_trainable_params': []}}},
  '_parameters': {'demo': {'data': 'demo', 'requires_opt': True}},
  'training': False}}

In [11]:
from utils.serialization import save_json

save_json(doc.to_dict(), "doc.json")

In [12]:
doc.state_dict()

OrderedDict([('demo', Parameter: demo)])

In [13]:
doc.call("What is the best treatment for a cold?")

2024-06-09 20:25:22 - INFO - [generator.py:196:call] - prompt_kwargs: {'input_str': 'What is the best treatment for a cold?'}
2024-06-09 20:25:22 - INFO - [generator.py:197:call] - model_kwargs: {}
2024-06-09 20:25:22 - INFO - [openai_client.py:122:call] - api_kwargs: {'model': 'gpt-3.5-turbo', 'messages': [{'role': 'system', 'content': '<SYS> You are a doctor </SYS> User: What is the best treatment for a cold?'}]}
2024-06-09 20:25:25 - INFO - [_client.py:1026:_send_single_request] - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-06-09 20:25:26 - INFO - [generator.py:205:call] - output: GeneratorOutput(data='As a doctor, I recommend the following treatments for a cold:\n\n1. Rest: Get plenty of rest to allow your body to recover and fight off the cold virus.\n\n2. Stay hydrated: Drink plenty of fluids such as water, herbal teas, and clear broths to help keep your throat moist and loosen congestion.\n\n3. Over-the-counter medications: Consider takin

'As a doctor, I recommend the following treatments for a cold:\n\n1. Rest: Get plenty of rest to allow your body to recover and fight off the cold virus.\n\n2. Stay hydrated: Drink plenty of fluids such as water, herbal teas, and clear broths to help keep your throat moist and loosen congestion.\n\n3. Over-the-counter medications: Consider taking over-the-counter medications such as acetaminophen or ibuprofen to help reduce fever and relieve aches and pains. Decongestants and antihistamines can also help alleviate nasal congestion and runny nose.\n\n4. Saline nasal drops or sprays: These can help relieve nasal congestion and improve breathing.\n\n5. Gargling with warm salt water: This can help soothe a sore throat.\n\n6. Humidifier: Using a humidifier in your room can help keep the air moist and ease congestion.\n\n7. Vitamin C and Zinc: Some studies suggest that vitamin C and zinc supplements may help reduce the duration and severity of a cold.\n\nRemember to consult with your healthc

In [14]:
from lightrag.core.component import FunComponent

def add_one(x):
    return x + 1

fun_component = FunComponent(add_one)
print(fun_component(1))  
print(type(fun_component))  

# output:
# 2
# <class 'core.component.FunComponent'>

2
<class 'lightrag.core.component.FunComponent'>


In [15]:
from lightrag.core.component import fun_to_component 

fun_component = fun_to_component(add_one)
print(fun_component(1))
print(type(fun_component))

# output:
# 2
# <class 'lightrag.core.component.AddOneComponent'>

2
<class 'lightrag.core.component.AddOneComponent'>


In [16]:
# use it as a decorator
@fun_to_component
def add_one(x):
    return x + 1

print(add_one(1))
print(type(add_one))

# output:
# 2
# <class 'lightrag.core.component.AddOneComponent'>

2
<class 'lightrag.core.component.AddOneComponent'>


In [17]:
from lightrag.core.component import Sequential

@fun_to_component
def enhance_query(query:str) -> str:
    return query + "Please be concise and only list the top treatments."

seq = Sequential(enhance_query, doc)

query = "What is the best treatment for headache?"
print(seq(query))

2024-06-09 20:25:26 - INFO - [generator.py:196:call] - prompt_kwargs: {'input_str': 'What is the best treatment for headache?Please be concise and only list the top treatments.'}
2024-06-09 20:25:26 - INFO - [generator.py:197:call] - model_kwargs: {}
2024-06-09 20:25:26 - INFO - [openai_client.py:122:call] - api_kwargs: {'model': 'gpt-3.5-turbo', 'messages': [{'role': 'system', 'content': '<SYS> You are a doctor </SYS> User: What is the best treatment for headache?Please be concise and only list the top treatments.'}]}
2024-06-09 20:25:26 - INFO - [_client.py:1026:_send_single_request] - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-06-09 20:25:26 - INFO - [generator.py:205:call] - output: GeneratorOutput(data='1. Over-the-counter pain medications such as ibuprofen or acetaminophen\n2. Getting enough rest and staying hydrated\n3. Using cold or warm compress on the forehead\n4. Practicing relaxation techniques like deep breathing or meditation', er

In [18]:
seq

Sequential(
  (0): EnhanceQueryComponent()
  (1): DocQA(
    (doc): Generator(
      model_kwargs={'model': 'gpt-3.5-turbo'}, model_type=ModelType.LLM
      (prompt): Prompt(template: <SYS> You are a doctor </SYS> User: {{input_str}}, prompt_variables: ['input_str'])
      (model_client): OpenAIClient()
    )
  )
)

Here is our first approach to build a single task with multiple generators and call each conditionally.

In [19]:

class ChatBotWithRouter(Component):
    def __init__(self):
        super().__init__()
        model_1_kwargs = {
            "model": "gpt-3.5-turbo",
        }
        model_2_kwargs = {"model": "llama3-8b-8192"}
        self.doc = Generator(
            template=template_doc,
            model_client=OpenAIClient(),
            model_kwargs=model_1_kwargs,
        )
        self.lawyer = Generator(
            template=template_law,
            model_client=GroqAPIClient(),
            model_kwargs=model_2_kwargs,
        )
        self.router_choices = {
            "doctor": self.create_generator_signature(self.doc),
            "lawyer": self.create_generator_signature(self.lawyer),
            "other": "Choose me the question does not apply to other choices.",
        }
        print(self.router_choices)

        self.router = Generator(
            template=template_router,
            model_client=OpenAIClient(),
            model_kwargs=model_1_kwargs,
        )

    def call(self, query: str) -> str:
        choice = self.router(
            prompt_kwargs={"input_str": query, "choices": self.router_choices}
        ).data
        if choice == "doctor":
            return self.doc(prompt_kwargs={"input_str": query}).data
        elif choice == "lawyer":
            return self.lawyer(prompt_kwargs={"input_str": query}).data
        else:
            return "Sorry, I cannot help you with that."

    def create_generator_signature(self, generator: Generator):
        template = generator.template
        pattern = r"<SYS>(.*?)</SYS>"

        matches = re.findall(pattern, template)
        for match in matches:
            print("Content between <SYS> tags:", match)
            return match

Initiate the task component, and print the task details.

In [20]:
task = ChatBotWithRouter()
task

2024-06-09 20:25:26 - INFO - [prompt_builder.py:82:__init__] - Prompt has variables: ['input_str']
2024-06-09 20:25:26 - INFO - [prompt_builder.py:82:__init__] - Prompt has variables: ['input_str']
Content between <SYS> tags:  You are a doctor 
Content between <SYS> tags:  You are a lawyer 
{'doctor': ' You are a doctor ', 'lawyer': ' You are a lawyer ', 'other': 'Choose me the question does not apply to other choices.'}
2024-06-09 20:25:26 - INFO - [prompt_builder.py:82:__init__] - Prompt has variables: ['input_str', 'choices']


ChatBotWithRouter(
  (doc): Generator(
    model_kwargs={'model': 'gpt-3.5-turbo'}, model_type=ModelType.LLM
    (prompt): Prompt(template: <SYS> You are a doctor </SYS> User: {{input_str}}, prompt_variables: ['input_str'])
    (model_client): OpenAIClient()
  )
  (lawyer): Generator(
    model_kwargs={'model': 'llama3-8b-8192'}, model_type=ModelType.LLM
    (prompt): Prompt(template: <SYS> You are a lawyer </SYS> User: {{input_str}}, prompt_variables: ['input_str'])
    (model_client): GroqAPIClient()
  )
  (router): Generator(
    model_kwargs={'model': 'gpt-3.5-turbo'}, model_type=ModelType.LLM
    (prompt): Prompt(
      template: <SYS> You are a router who will route a user question to the right generator.
                  Here are your choices in form of key: value pairs:
                   {% for key, value in choices.items() %}
                      {{ key }}: {{ value }}
                   {% endfor %}
                  Output the key of your choice.
                  </SYS> 

Call the task with a query

In [21]:
query = "I have a legal question"
print(task(query))

2024-06-09 20:25:26 - INFO - [generator.py:196:call] - prompt_kwargs: {'input_str': 'I have a legal question', 'choices': {'doctor': ' You are a doctor ', 'lawyer': ' You are a lawyer ', 'other': 'Choose me the question does not apply to other choices.'}}
2024-06-09 20:25:26 - INFO - [generator.py:197:call] - model_kwargs: {}
2024-06-09 20:25:26 - INFO - [openai_client.py:122:call] - api_kwargs: {'model': 'gpt-3.5-turbo', 'messages': [{'role': 'system', 'content': '<SYS> You are a router who will route a user question to the right generator.\n            Here are your choices in form of key: value pairs:\n                doctor:  You are a doctor \n                lawyer:  You are a lawyer \n                other: Choose me the question does not apply to other choices.\n            Output the key of your choice.\n            </SYS> User question: I have a legal question\n            You:'}]}
2024-06-09 20:25:27 - INFO - [_client.py:1026:_send_single_request] - HTTP Request: POST https:

Now, let's separate this into multiple subtasks and ``chain`` them using the ``Sequential`` container.

First, the router task which will takes a dictionary of choices and return the selected key. In addition, we use ``_extra_repr`` to improve the default string representation of the task.

As ``Sequential`` will pass the output of one task to the next using positional arguments, we return whatever is needed to the next task in a dictionary.

In [22]:
from typing import Dict
class Router(Component):
    def __init__(self, choices: Dict[str, str] = {}):
        super().__init__()
        self.choices = choices
        self.router = Generator(
            template=template_router,
            model_client=OpenAIClient(),
            model_kwargs={"model": "gpt-3.5-turbo"},
        )

    def call(self, query: str) -> str:
        prompt_kwargs = {"input_str": query, "choices": self.choices}
        choice =  self.router(prompt_kwargs=prompt_kwargs).data
        return {"choice": choice, "query": query}
    
    def _extra_repr(self):
        return f"Choices: {self.choices}, "

See the structure of router task.

In [23]:
r = Router()
r

2024-06-09 20:25:27 - INFO - [prompt_builder.py:82:__init__] - Prompt has variables: ['input_str', 'choices']


Router(
  Choices: {}, 
  (router): Generator(
    model_kwargs={'model': 'gpt-3.5-turbo'}, model_type=ModelType.LLM
    (prompt): Prompt(
      template: <SYS> You are a router who will route a user question to the right generator.
                  Here are your choices in form of key: value pairs:
                   {% for key, value in choices.items() %}
                      {{ key }}: {{ value }}
                   {% endfor %}
                  Output the key of your choice.
                  </SYS> User question: {{input_str}}
                  You:
                  , prompt_variables: ['input_str', 'choices']
    )
    (model_client): OpenAIClient()
  )
)

Now, lets build another subtask which handles the chat depending on the selected key from the router task.
As the router task returns a dictionary, we will make our input dictionary type that parses the ``choice`` and ``query`` key value pairs.

In [24]:
class Chat(Component):
    def __init__(self):
        super().__init__()
        self.doc = Generator(
            template=template_doc,
            model_client=OpenAIClient(),
            model_kwargs={"model": "gpt-3.5-turbo"},
        )
        self.lawyer = Generator(
            template=template_law,
            model_client=GroqAPIClient(),
            model_kwargs={"model": "llama3-8b-8192"},
        )
    # to chain together just to make sure the output can be directly passed to the next as input
    def call(self, input: Dict[str, str]) -> Dict[str, str]:
        choice = input.get("choice", None)
        query = input.get("query", None)
        if choice == "doctor":
            return self.doc(prompt_kwargs={"input_str": query}).data
        elif choice == "lawyer":
            return self.lawyer(prompt_kwargs={"input_str": query}).data
        else:
            return "Sorry, I am not able to help you with that."



In [25]:
chat = Chat()
chat

2024-06-09 20:25:27 - INFO - [prompt_builder.py:82:__init__] - Prompt has variables: ['input_str']
2024-06-09 20:25:27 - INFO - [prompt_builder.py:82:__init__] - Prompt has variables: ['input_str']


Chat(
  (doc): Generator(
    model_kwargs={'model': 'gpt-3.5-turbo'}, model_type=ModelType.LLM
    (prompt): Prompt(template: <SYS> You are a doctor </SYS> User: {{input_str}}, prompt_variables: ['input_str'])
    (model_client): OpenAIClient()
  )
  (lawyer): Generator(
    model_kwargs={'model': 'llama3-8b-8192'}, model_type=ModelType.LLM
    (prompt): Prompt(template: <SYS> You are a lawyer </SYS> User: {{input_str}}, prompt_variables: ['input_str'])
    (model_client): GroqAPIClient()
  )
)

Now, lets chain the router and the chat task using the ``Sequential`` container into a runnable pipeline.

In [26]:
class QAWithRouter(Component):
    def __init__(self):
        super().__init__()
        self.router = Router(choices={"doctor": "Doctor", "lawyer": "Lawyer", "other": "Other"})
        self.chat = Chat()
        self.pipeline = Sequential(self.router, self.chat)

    def call(self, query: str) -> str:
        return self.pipeline(query)

In [27]:
qa = QAWithRouter()
qa

2024-06-09 20:25:27 - INFO - [prompt_builder.py:82:__init__] - Prompt has variables: ['input_str', 'choices']
2024-06-09 20:25:27 - INFO - [prompt_builder.py:82:__init__] - Prompt has variables: ['input_str']
2024-06-09 20:25:27 - INFO - [prompt_builder.py:82:__init__] - Prompt has variables: ['input_str']


QAWithRouter(
  (router): Router(
    Choices: {'doctor': 'Doctor', 'lawyer': 'Lawyer', 'other': 'Other'}, 
    (router): Generator(
      model_kwargs={'model': 'gpt-3.5-turbo'}, model_type=ModelType.LLM
      (prompt): Prompt(
        template: <SYS> You are a router who will route a user question to the right generator.
                    Here are your choices in form of key: value pairs:
                     {% for key, value in choices.items() %}
                        {{ key }}: {{ value }}
                     {% endfor %}
                    Output the key of your choice.
                    </SYS> User question: {{input_str}}
                    You:
                    , prompt_variables: ['input_str', 'choices']
      )
      (model_client): OpenAIClient()
    )
  )
  (chat): Chat(
    (doc): Generator(
      model_kwargs={'model': 'gpt-3.5-turbo'}, model_type=ModelType.LLM
      (prompt): Prompt(template: <SYS> You are a doctor </SYS> User: {{input_str}}, prompt_variables

In [28]:
qa("I have a legal question")

2024-06-09 20:25:27 - INFO - [generator.py:196:call] - prompt_kwargs: {'input_str': 'I have a legal question', 'choices': {'doctor': 'Doctor', 'lawyer': 'Lawyer', 'other': 'Other'}}
2024-06-09 20:25:27 - INFO - [generator.py:197:call] - model_kwargs: {}
2024-06-09 20:25:27 - INFO - [openai_client.py:122:call] - api_kwargs: {'model': 'gpt-3.5-turbo', 'messages': [{'role': 'system', 'content': '<SYS> You are a router who will route a user question to the right generator.\n            Here are your choices in form of key: value pairs:\n                doctor: Doctor\n                lawyer: Lawyer\n                other: Other\n            Output the key of your choice.\n            </SYS> User question: I have a legal question\n            You:'}]}
2024-06-09 20:25:28 - INFO - [_client.py:1026:_send_single_request] - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-06-09 20:25:28 - INFO - [generator.py:205:call] - output: GeneratorOutput(data='other', 

'Sorry, I am not able to help you with that.'

# a router and two generators
# every single component might need a signature.
# TODO: LLM for single choices